In [ ]:
! pip install transformers

In [ ]:
import random
import torch
import numpy as np

from google.colab import drive
from PIL import Image
from os import path
from tqdm import tqdm
from transformers import ViTFeatureExtractor, ViTForImageClassification

In [ ]:
drive.mount('/content/drive')

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

In [ ]:
IMG_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/runwaysd_images'

In [ ]:
PROMPT_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes.txt'
OPPOSITE_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes_opposites.txt'
SAVE_PATH  = f'/content/drive/My Drive/CLIP_Facial_Impressions/sdxl_images'

In [ ]:
prompt_attributes = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(PROMPT_PATH).read().split('\n') if line}
opposite_prompts = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(OPPOSITE_PATH).read().split('\n') if line}

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-large-patch32-384')
model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

In [ ]:
for attribute in prompt_attributes.keys():
  positive = [f'{attribute}_positive_{i}.png' for i in range(25)]
  negative = [f'{attribute}_negative_{i}.png' for i in range(25)]
  imgs = positive + negative

  vecs = []

  try:
    for img in tqdm(imgs):
      image = Image.open(path.join(IMG_PATH, img))
      inputs = feature_extractor(images=image, return_tensors="pt").to(device)
      with torch.no_grad():
        outputs = model(**inputs,output_hidden_states=True)
        vec = outputs.hidden_states[-1].mean(axis=1)
        vecs.append(vec)

    final_vecs = torch.stack(vecs)
    final_vecs = final_vecs.squeeze(1)
    final_vecs.size()
    final_vecs = final_vecs.to('cpu')

    np_vecs = final_vecs.numpy()
    np.save(f'/content/drive/My Drive/CLIP_Facial_Impressions/sdxl_vit_embs/{attribute}_runwaysd_vit_vecs.npy', np_vecs)
  except:
    continue